In [1]:
import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from srgan3 import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# !TF_ENABLE_GPU_GARBAGE_COLLECTION=false

In [3]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [4]:
r = 3
input_dir = f'data/train/{r}/low/'
tgt_dir = f'data/train/{r}/high/'

input_dir_val = f'data/val/{r}/low/'
tgt_dir_val = f'data/val/{r}/high/'


test_file = 'UBMk30rjy0o_17675_42.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_path = 'data/out/' + test_file
test_test_path = 'data/test/' + test_file

vgg_path = 'models/vgg16_notop.hdf5'

input_shape = (45, 80, 3)
tgt_shape = (135, 240, 3)

batch_size = 16

In [5]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)
gen_valid = ImageDataGenerator(input_dir_val, tgt_dir_val, batch_size)

/home/keras/srgan3.py:117: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("ou...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')
/home/keras/srgan3.py:149: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="discriminator", inputs=Tensor("in..., outputs=Tensor("de...)`
  self.shared_discriminator = Network(input=layers[0], output=layers[-1], name='discriminator')
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
for e in range(20001):
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 100 == 0:
        print('epoch', e)
        srgan.valid(X_low, X_high)
        
        test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
        test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
        test_out_path = 'data/out/'  + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:0.5958585143089294, d_loss_fake:0.8214282989501953, vgg_loss:5.264488697052002
epoch 100
d_loss_real:0.7360556125640869, d_loss_fake:0.6874650716781616, vgg_loss:1.920102596282959
epoch 200
d_loss_real:0.6226239204406738, d_loss_fake:0.873725414276123, vgg_loss:1.197772741317749
epoch 300
d_loss_real:0.6665489673614502, d_loss_fake:1.0209064483642578, vgg_loss:2.0929782390594482
epoch 400
d_loss_real:0.8419559001922607, d_loss_fake:1.090919017791748, vgg_loss:1.8032357692718506
epoch 500
d_loss_real:0.5283955335617065, d_loss_fake:1.0479382276535034, vgg_loss:2.8714230060577393
epoch 600
d_loss_real:0.5303426384925842, d_loss_fake:0.9688657522201538, vgg_loss:1.071855902671814
epoch 700
d_loss_real:0.4945247173309326, d_loss_fake:1.0289337635040283, vgg_loss:1.6384694576263428
epoch 800
d_loss_real:0.472786009311676, d_loss_fake:1.290165901184082, vgg_loss:1.4195904731750488
epoch 900
d_loss_real:0.7184351682662964, d_loss_fake:1.176013708114624, vgg_loss:1.31461524

KeyboardInterrupt: 

In [9]:
srgan.generator.save('models/20200513_generator.h5', include_optimizer=False)